In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
!pip install accelerate

In [ ]:
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True,padding='max_length', max_length=256)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
dataset = load_dataset("data")

tokenized_data = dataset.map(preprocess_function, batched=True, batch_size=1000, num_proc=4)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length")
accuracy = evaluate.load("accuracy")

id2label = {0: "1 звезда", 1: "2 звезды", 2: "3 звезды", 3: "4 звезды", 4: "5 звёзд"}
label2id = {"1 звезда": 0, "2 звезды": 1, "3 звезды": 2, "4 звезды": 3, "5 звёзд": 4}

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map (num_proc=4):   0%|          | 0/650000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased",
    num_labels=5,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="yelp_review_classifier",
    learning_rate=2e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,
    optim="adamw_torch",
    save_total_limit=2,
    dataloader_num_workers=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save("model")
trainer.push_to_hub()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Epoch,Training Loss,Validation Loss,Accuracy
1,0.997100,1.087061,0.518580
2,0.863200,0.902566,0.606320


AttributeError: 'Trainer' object has no attribute 'save'

In [ ]:
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ssifier/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...433533.0149c859e4f4.624.0: 100%|##########| 23.7kB / 23.7kB            

  ...ssifier/model.safetensors:   6%|6         | 16.7MB /  268MB            

CommitInfo(commit_url='https://huggingface.co/kertys/yelp_review_classifier/commit/b0ae9662584bb0865a3b15c3d14a8c8513edef6e', commit_message='End of training', commit_description='', oid='b0ae9662584bb0865a3b15c3d14a8c8513edef6e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kertys/yelp_review_classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='kertys/yelp_review_classifier'), pr_revision=None, pr_num=None)